#### Simple notebook to run model tests only

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Load inn datasets
X_test  = pd.read_parquet('data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('data/prepared_datasets/only_y_cleaned/Y_train.parquet')

## Catboost

In [3]:
from catboost import Pool, CatBoostRegressor

#Create a pool of data
train_pool = Pool(X_train, y_train, cat_features=["location"])
test_pool = Pool(X_test, cat_features=["location"]) 

cat_feature = ["location"]

#init model and fit it
catboost_model = CatBoostRegressor(cat_features=cat_feature, iterations=429, depth=10)
catboost_model.fit(train_pool)

Learning rate set to 0.166715
0:	learn: 663.2513911	total: 361ms	remaining: 2m 34s
1:	learn: 579.9517057	total: 629ms	remaining: 2m 14s
2:	learn: 514.0364811	total: 905ms	remaining: 2m 8s
3:	learn: 459.9948145	total: 1.17s	remaining: 2m 4s
4:	learn: 416.8667786	total: 1.41s	remaining: 1m 59s
5:	learn: 382.8083926	total: 1.67s	remaining: 1m 57s
6:	learn: 356.8847141	total: 1.91s	remaining: 1m 55s
7:	learn: 336.6649663	total: 2.15s	remaining: 1m 52s
8:	learn: 321.1503127	total: 2.4s	remaining: 1m 52s
9:	learn: 308.7478537	total: 2.65s	remaining: 1m 51s
10:	learn: 299.2322334	total: 2.91s	remaining: 1m 50s
11:	learn: 291.8964728	total: 3.17s	remaining: 1m 50s
12:	learn: 285.8105985	total: 3.4s	remaining: 1m 48s
13:	learn: 281.8254101	total: 3.61s	remaining: 1m 47s
14:	learn: 278.3287778	total: 3.84s	remaining: 1m 45s
15:	learn: 275.8470609	total: 4.06s	remaining: 1m 44s
16:	learn: 272.7961490	total: 4.28s	remaining: 1m 43s
17:	learn: 270.9094966	total: 4.49s	remaining: 1m 42s
18:	learn: 2

#### Catboost for A and (B and C)

#### Local Test for all data in 1 dataset

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
ts = TimeSeriesSplit(n_splits = 10)
sklearn_cv_result = cross_val_score(catboost_model, X_train, y_train, cv=ts, scoring='neg_mean_absolute_error')

Learning rate set to 0.114141
0:	learn: 982.6346797	total: 72.3ms	remaining: 31s
1:	learn: 907.0005882	total: 143ms	remaining: 30.6s
2:	learn: 844.4707165	total: 220ms	remaining: 31.2s
3:	learn: 786.8797230	total: 296ms	remaining: 31.4s
4:	learn: 737.5797768	total: 379ms	remaining: 32.1s
5:	learn: 694.0643657	total: 471ms	remaining: 33.2s
6:	learn: 656.7459384	total: 556ms	remaining: 33.5s
7:	learn: 625.6347660	total: 645ms	remaining: 34s
8:	learn: 597.2820058	total: 721ms	remaining: 33.6s
9:	learn: 571.5928289	total: 794ms	remaining: 33.3s
10:	learn: 551.5221254	total: 863ms	remaining: 32.8s
11:	learn: 533.5259982	total: 922ms	remaining: 32s
12:	learn: 516.9246834	total: 989ms	remaining: 31.7s
13:	learn: 504.2969229	total: 1.05s	remaining: 31s
14:	learn: 490.9389784	total: 1.11s	remaining: 30.6s
15:	learn: 481.4096977	total: 1.17s	remaining: 30.3s
16:	learn: 471.5388134	total: 1.24s	remaining: 30s
17:	learn: 461.3309008	total: 1.3s	remaining: 29.7s
18:	learn: 452.2617921	total: 1.36s	

In [6]:
np.mean(sklearn_cv_result)

-156.68724296087728

In [24]:
#make predictions
predictions = pd.DataFrame(catboost_model.predict(test_pool))
print(predictions)

              0
0    -13.676720
1    -13.227421
2    -13.369704
3    -14.783190
4    -16.363580
...         ...
8635   6.265602
8636   5.405113
8637   5.570684
8638   7.107676
8639   7.706747

[8640 rows x 1 columns]


In [32]:
#Local testing
from catboost import cv
cv_results = cv(train_pool, catboost_model.get_params(), fold_count=5, type="TimeSeries")

Training on fold [0/5]
0:	learn: 1222.0631901	test: 1399.5899229	best: 1399.5899229 (0)	total: 7.78ms	remaining: 7.77s
1:	learn: 1193.6916953	test: 1368.7883766	best: 1368.7883766 (1)	total: 15.3ms	remaining: 7.65s
2:	learn: 1165.8178325	test: 1338.2526015	best: 1338.2526015 (2)	total: 22.1ms	remaining: 7.34s
3:	learn: 1138.8767039	test: 1308.3853756	best: 1308.3853756 (3)	total: 29.2ms	remaining: 7.26s
4:	learn: 1112.4572171	test: 1279.1558277	best: 1279.1558277 (4)	total: 39.3ms	remaining: 7.83s
5:	learn: 1086.8450406	test: 1251.0729947	best: 1251.0729947 (5)	total: 47.8ms	remaining: 7.92s
6:	learn: 1061.6202309	test: 1223.3197364	best: 1223.3197364 (6)	total: 56.4ms	remaining: 8s
7:	learn: 1037.6937050	test: 1197.0781976	best: 1197.0781976 (7)	total: 66.8ms	remaining: 8.28s
8:	learn: 1014.5862402	test: 1171.4466301	best: 1171.4466301 (8)	total: 74.5ms	remaining: 8.2s
9:	learn: 992.1449623	test: 1147.0548418	best: 1147.0548418 (9)	total: 82.6ms	remaining: 8.18s
10:	learn: 971.4391128

In [36]:
1/cv_results['test-RMSE-mean'].min()

0.0029318377232400933

## Setting up optuna for catboost

In [ ]:
import optuna 

def objective(trial, X_train, Y_train):
    max_depth = trial.suggest_int("max_depth", 5, 20)
    n_estimators = trial.suggest_int("n_estimators", 20, 40)
    
    rf_regressor = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    scores = cross_val_score(rf_regressor, X_train, Y_train, cv=2)

    return np.min([np.mean(scores), np.median([scores])])


study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train_all_rf, Y_train_all), n_trials=2)

In [38]:
import optuna

In [39]:
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 2500),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    model = CatBoostRegressor(**params, silent=True)
    ts = TimeSeriesSplit(n_splits = 10)
    scores = cross_val_score(catboost_model, X_train, y_train, cv=ts, scoring='neg_mean_absolute_error')

    return np.min([np.mean(scores), np.median([scores])])
    
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=10)

[I 2023-11-03 18:48:05,396] A new study created in memory with name: no-name-0f35ec2d-b9d2-468a-8b93-0da14cf9b03b


Learning rate set to 0.057363
0:	learn: 1023.7802647	total: 4.15ms	remaining: 4.14s
1:	learn: 982.9867698	total: 8.25ms	remaining: 4.12s
2:	learn: 944.4994761	total: 12ms	remaining: 3.99s
3:	learn: 909.4655214	total: 15.7ms	remaining: 3.92s
4:	learn: 876.6432905	total: 19.5ms	remaining: 3.88s
5:	learn: 844.5475322	total: 23.6ms	remaining: 3.92s
6:	learn: 816.2036267	total: 27.5ms	remaining: 3.9s
7:	learn: 789.8444578	total: 31.1ms	remaining: 3.85s
8:	learn: 764.6825354	total: 34.3ms	remaining: 3.77s
9:	learn: 741.3659895	total: 38.4ms	remaining: 3.8s
10:	learn: 720.0237180	total: 41.8ms	remaining: 3.76s
11:	learn: 699.7181581	total: 45.2ms	remaining: 3.72s
12:	learn: 681.6554707	total: 48.7ms	remaining: 3.7s
13:	learn: 664.9728651	total: 52.6ms	remaining: 3.7s
14:	learn: 648.6787826	total: 56.4ms	remaining: 3.7s
15:	learn: 633.5666581	total: 60ms	remaining: 3.69s
16:	learn: 620.4308943	total: 63.5ms	remaining: 3.67s
17:	learn: 608.7406909	total: 67.2ms	remaining: 3.67s
18:	learn: 596.6

[W 2023-11-03 18:49:27,591] Trial 0 failed with parameters: {'iterations': 1123, 'learning_rate': 0.08299309605027054, 'depth': 2, 'subsample': 0.15045164706744013, 'colsample_bylevel': 0.8394953860130746, 'min_data_in_leaf': 60} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "c:\Users\Shahl\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Shahl\AppData\Local\Temp\ipykernel_8756\1178597384.py", line 18, in <lambda>
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100)
  File "C:\Users\Shahl\AppData\Local\Temp\ipykernel_8756\1178597384.py", line 13, in objective
    scores = cross_val_score(catboost_model, X_train, y_train, cv=ts, scoring='neg_mean_absolute_error')
  File "c:\Users\Shahl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 515, in c

750:	learn: 251.0090350	total: 36.2s	remaining: 12s
751:	learn: 250.9718831	total: 36.3s	remaining: 12s
752:	learn: 250.8886533	total: 36.3s	remaining: 11.9s


KeyboardInterrupt: 

## Preparing submission

In [25]:
def prepare_submission(predictions, X_test):
    index_df = X_test.index.to_frame()
    out_pd = pd.concat([index_df.reset_index(drop=True), predictions.reset_index(drop=True)], axis=1)
    out_pd = out_pd.rename(columns={0: 'prediction', 'date_forecast': 'time'})
    print(X_test.info())
    print(out_pd.info())
    out_pd['location'] = X_test['location'].reset_index(drop=True)
    out_pd.set_index('time', inplace=True)
    return out_pd

def merge_with_sample(out_pd):
    test = pd.read_csv('data/test.csv')
    test.time = pd.to_datetime(test.time)
    sample_submission = pd.read_csv('data/sample_submission.csv')
    test.set_index('time', inplace=True)
    
    merged_df = test.reset_index().merge(out_pd.reset_index(), on=['time', 'location'], how='left', suffixes=('_original', '_new'))
    merged_df['prediction_new'] = merged_df['prediction_new'].combine_first(merged_df['prediction_original'])
    merged_df.drop('prediction_original', axis=1, inplace=True)
    merged_df.rename(columns={'prediction_new': 'prediction'}, inplace=True)
    return sample_submission[['id']].merge(merged_df[['id', 'prediction']], on='id', how='left')


out_pd = prepare_submission(predictions, X_test)
sample_submission = merge_with_sample(out_pd)
sample_submission.to_csv('submission.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8640 entries, 2023-05-01 00:00:00 to 2023-07-03 23:45:00
Data columns (total 46 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   absolute_humidity_2m:gm3        8640 non-null   float32
 1   air_density_2m:kgm3             8640 non-null   float32
 2   ceiling_height_agl:m            8635 non-null   float32
 3   clear_sky_energy_1h:J           8640 non-null   float32
 4   clear_sky_rad:W                 8640 non-null   float32
 5   cloud_base_agl:m                8640 non-null   float32
 6   dew_or_rime:idx                 8640 non-null   float32
 7   dew_point_2m:K                  8640 non-null   float32
 8   diffuse_rad:W                   8640 non-null   float32
 9   diffuse_rad_1h:J                8640 non-null   float32
 10  direct_rad:W                    8640 non-null   float32
 11  direct_rad_1h:J                 8640 non-null   float32
 12